## Task 1. Stateful wordcount

In this task you're receiving batches in real time through DStream. You need to create the Wordcount program with the saving and updating of the state after each batch. 

You have to print the TOP-10 most popular words in the input dataset and its quantity. 

There are several points for this task:

1) You have to print the data only at the end. The criteria is if you have received the first empty RDD, the stream is finished. At this moment you have to print the result and stop the context.

2) You may split the line with using $flatMap$ method in DStream.

3) In this task you need to filter out short words (with length less than 4). For this aim you may use $filter$ method in DStream.

4) Remember that you should use string lowercase. You may use $map$ method in DStream to transform words to such case.

5) You may use  $reduceByKey$ in Dstream to merge the tuples with the same key by summing the word count value.

6) In this task, you need to be able to maintain the state across the batches. You may use the $updateStateByKey()$ method, which provides an access to the state variable and helps you to implement the "stateful" approach. You can update the current state with the results of every batch.

You may find more useful methods in the following sources:

* Book "Learning Spark: Lightning-Fast Big Data Analysis" by Holden Karau.

* [Spark Streaming documentation](https://spark.apache.org/docs/latest/streaming-programming-guide.html)

* [PySpark Streaming documentation](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark-streaming-module) 

* [PySpark Streaming examples](https://github.com/apache/spark/tree/master/examples/src/main/python/streaming)


Here you can find the starter for the main steps of the task. You can use other methods to get the solution.

In [1]:
import os
import time
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

**NB.** Please don't change the cell below. It is used for emulation realtime batch arriving. But figure out the code, it will help you when you work with real SparkStreaming applications.

In [2]:
# Preparing SparkContext
sc = SparkContext(master='local[4]')

# Preparing batches with the input data
DATA_PATH = "/data/wiki/en_articles_streaming"

batches = [sc.textFile(os.path.join(DATA_PATH, path)) for path in os.listdir(DATA_PATH)]

# Creating Dstream to emulate realtime data generating
BATCH_TIMEOUT = 5  # Timeout between batch generation
ssc = StreamingContext(sc, BATCH_TIMEOUT)
dstream = ssc.queueStream(rdds=batches)

There are 2 flags used in this task.
* The `finished` flag indicates if the current RDD is empty.
* The `printed` one indicates that the result has been printed and SparkStreaming context can be stopped.

For filtering out punctuation and other junk symbols use this pattern: `re.split("\W*\s+\W*", line.strip(), flags=re.UNICODE)`

**NB**. Spark transformations work in a lazy mode. When the transformation is called, it doesn't execute really. It just saves in the computational DAG. All the transformations will be executed when the action will be called. Let's look at `print_only_at_the_end()` function. The action will be called only when the stream will be finished. So in this moment  Spark will execute all the transformations. This will lead to container's overflow if the dataset is really big. So if you faced the error like `Container killed by YARN for exceeding memory limits`, call some action before `if` clause in this function.

In [4]:
finished = False
printed = False


def set_ending_flag(rdd):
    global finished
    if rdd.isEmpty():
        finished = True


def print_only_at_the_end(rdd):
    global printed
    # Type your code for printing the sorted data from the stream in a loop
    #     print(rdd.takeOrdered(20,lambda x:-x[1]))
    if finished and not printed:
        # Type your code for printing the sorted data from the stream in a loop
        print("final answer")
        print(rdd.takeOrdered(20,lambda x:-x[1]))
        printed = True


# If we have received an empty rdd, the stream is finished.
# So print the result and stop the context.
dstream.foreachRDD(set_ending_flag)

In [5]:
# Type your code for data processing and aggregation here
def clean(word):
    new_word = ""
    for ch in word:
        if 'a'<=ch<='z' or '0'<=ch<='9':
            new_word +=ch
    return new_word
def agg(ls,old):
    return (old or 0)+sum(ls)
dstream.flatMap(lambda x: x.lower().split()).map(lambda x: (clean(x),1)) \
    .filter(lambda x: len(x[0])>=4) \
    .reduceByKey(lambda x,y:x+y) \
    .updateStateByKey(agg) \
    .foreachRDD(print_only_at_the_end)
# dstream.map(...) \
#        .filter(...) ...

**NB.** Please don't change the cell below. It is used for stopping SparkStreaming context and Spark context when the stream finished.

In [ ]:
ssc.checkpoint('./checkpoin{}'.format(time.strftime("%Y_%m_%d_%H_%M_%s", time.gmtime())))  # checkpoint for storing current state        
ssc.start()
while not printed:
    pass
ssc.stop()  # when the result was printed, stop SparkStreaming context
sc.stop()  # stop Spark context to be able restart the code without restarting the kernel

final answer
[('that', 81573), ('with', 79560), ('from', 58201), ('which', 42199), ('this', 38254), ('were', 34496), ('also', 31575), ('have', 29871), ('their', 24579), ('other', 23538), ('first', 22239), ('they', 20579), ('been', 20466), ('such', 19602), ('more', 18579), ('used', 18233), ('some', 18100), ('most', 17610), ('american', 17218), ('after', 16170)]


Here you can see a part of an output on the sample dataset:

```
...
which 42198
this 38252
were 34403
...
```

Of course, the numbers may be different but not very much (the error about 2% will be accepted).